## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-15-03-02-01 +0000,bbc,"Animals react to secret sounds from plants, sa...",https://www.bbc.com/news/articles/c8e4860n9rpo
1,2025-07-15-02-49-09 +0000,nypost,Flash flooding across tri-state area prompts r...,https://nypost.com/2025/07/14/us-news/flash-fl...
2,2025-07-15-02-28-35 +0000,bbc,China growth slows as trade turmoil weighs on ...,https://www.bbc.com/news/articles/c20r461g61xo
3,2025-07-15-02-15-17 +0000,nypost,Three Democrats win double-digit support in ea...,https://nypost.com/2025/07/14/us-news/three-de...
4,2025-07-15-02-08-31 +0000,nyt,Trump Reluctantly Comes Around to Backing Ukra...,https://www.nytimes.com/2025/07/14/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2311/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
39,trump,16
44,ukraine,7
51,russia,6
92,would,5
114,tariff,5


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
59,2025-07-14-14-48-00 +0000,wsj,Trump is threatening to ramp up the U.S. trade...,https://www.wsj.com/economy/trade/european-uni...,48
13,2025-07-15-00-53-28 +0000,nypost,NATO chief warns Russia’s trading partners aft...,https://nypost.com/2025/07/14/us-news/nato-chi...,41
6,2025-07-15-02-01-34 +0000,bbc,Trump threatens Russia with tariffs while unve...,https://www.bbc.com/news/articles/czdv20v9lp1o,41
36,2025-07-14-20-35-11 +0000,nyt,Trump Threatens Russia With Sanctions and Vows...,https://www.nytimes.com/2025/07/14/us/politics...,36
5,2025-07-15-02-08-23 +0000,nyt,How Trump Changed His Tone on Putin and the Ru...,https://www.nytimes.com/2025/07/14/us/politics...,35


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
59,48,2025-07-14-14-48-00 +0000,wsj,Trump is threatening to ramp up the U.S. trade...,https://www.wsj.com/economy/trade/european-uni...
6,25,2025-07-15-02-01-34 +0000,bbc,Trump threatens Russia with tariffs while unve...,https://www.bbc.com/news/articles/czdv20v9lp1o
10,23,2025-07-15-01-25-19 +0000,nypost,Democratic city officers flocking to cop-frien...,https://nypost.com/2025/07/14/us-news/blue-cit...
25,20,2025-07-14-22-29-19 +0000,nypost,Founder of Cali pot farm where ICE rescued doz...,https://nypost.com/2025/07/14/us-news/founder-...
13,19,2025-07-15-00-53-28 +0000,nypost,NATO chief warns Russia’s trading partners aft...,https://nypost.com/2025/07/14/us-news/nato-chi...
69,18,2025-07-14-09-30-00 +0000,wsj,Food banks across the country were already str...,https://www.wsj.com/economy/food-banks-are-run...
27,18,2025-07-14-22-20-02 +0000,nypost,NYC beatdown victim turns tables and opens fir...,https://nypost.com/2025/07/14/us-news/nyc-beat...
32,16,2025-07-14-22-01-13 +0000,nypost,Bitter Texas woman tried poisoning newly engag...,https://nypost.com/2025/07/14/us-news/bitter-t...
14,16,2025-07-15-00-04-10 +0000,nypost,Teenage cleaner fatally sucked into meat grind...,https://nypost.com/2025/07/14/us-news/teenage-...
74,16,2025-07-14-06-08-00 +0000,wsj,"China’s exports grew at a faster clip in June,...",https://www.wsj.com/economy/trade/chinas-expor...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
